
# content
1. Introduction
2. connecting to POSTGRES
3. creating the db schema
4. testing bitemporal crud


## 1. Introduction

If You start this repo in gitpod, You can open this notebook in vscode and execute the code with julia and postgres up and running
https://www.gitpod.io/docs/

### 1.1. connecting to POSTGRES

In [ ]:
include("init.jl")


In [ ]:
using SearchLight
using SearchLightPostgreSQL
ENV["GENIE_ENV"] = "dev"
run(`psql -f sqlsnippets/droptables.sql`)
SearchLight.Configuration.load() |> SearchLight.connect
SearchLight.Migrations.create_migrations_table()
SearchLight.Migrations.up()

# 2 Starting with workflows

## 2.1 Start workflow 1 (the blue rectangle :=) )
inserting component 1 and subcomponent 1.1

In [ ]:
using Dates, Test, TimeZones
import BitemporalPostgres
using BitemporalPostgres

w1 = Workflow()
w1.tsw_validfrom = ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo")

t = TestDummyComponent()
tr = TestDummyComponentRevision(description = "blue")
ts = TestDummySubComponent(ref_super = t.id)
tsr = TestDummySubComponentRevision(description = "green")
create_entity!(w1)
create_component!(t, tr, w1)
println(tr)
create_subcomponent!(t, ts, tsr, w1)
w1.ref_history != Nothing
@test w1.is_committed == 0
@test w1.ref_version == tr.ref_validfrom
@test w1.ref_version == tsr.ref_validfrom

## 2.2 Commiting workflow 1

In [ ]:
commit_workflow!(w1)
@test w1.is_committed == 1

## Starting workflow 2 ( the yellow rectangle that shortens the blue one)
mutating component 1

In [ ]:
w2 = Workflow(
    ref_history = w1.ref_history,
    tsw_validfrom = ZonedDateTime(2015, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
tr2 = copy(tr)
tr2.description = "yellow"
update_entity!(w2)
update_component!(tr, tr2, w2)
println(tr2)
@test w2.ref_version == tr2.ref_validfrom
@test w2.ref_version == tr.ref_invalidfrom

## 2.3 Committing workflow 2

In [ ]:
commit_workflow!(w2)
@test w2.is_committed == 1

## 2.4 Starting workflow 3 (the red rectancle that shadows the yellow one)
Mutating component 1

In [ ]:
w3 = Workflow(
    ref_history = w1.ref_history,
    tsw_validfrom = ZonedDateTime(2014, 11, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
tr3 = copy(tr2)
tr3.description = "red"
update_entity!(w3)
update_component!(tr2, tr3, w3)
println(tr3)
@test w3.ref_version == tr3.ref_bvalidfrom
@test w3.ref_version == tr2.ref_invalidfrom

## 2.5 Committing workflow 3

In [ ]:
commit_workflow!(w3)
@test w3.is_committed == 1

# 3 Testing 
For each workflow 
* as of its intervals valid from points
* as of its intervals db valid from and 1 second before its world validfrom

In [ ]:
v1 = findversion(w1.ref_history, w1.tsdb_validfrom, w1.tsw_validfrom)

@test findcomponentrevision(TestDummyComponentRevision, t.id, v1)[1].description == "blue"

v2 = findversion(w2.ref_history, w2.tsdb_validfrom, w2.tsw_validfrom)

@test findcomponentrevision(TestDummyComponentRevision, t.id, v2)[1].description == "yellow"

v2a = findversion(w2.ref_history, w2.tsdb_validfrom, w2.tsw_validfrom - Dates.Second(1))

@test findcomponentrevision(TestDummyComponentRevision, t.id, v2a)[1].description == "blue"

v3 = findversion(w3.ref_history, w3.tsdb_validfrom, w3.tsw_validfrom)

@test findcomponentrevision(TestDummyComponentRevision, t.id, v3)[1].description == "red"

v3a = findversion(w3.ref_history, w3.tsdb_validfrom, w3.tsw_validfrom - Dates.Second(1))

@test findcomponentrevision(TestDummyComponentRevision, t.id, v3a)[1].description == "blue"


In [ ]:
v1 = findversion(w1.ref_history, w1.tsdb_validfrom, w1.tsw_validfrom)
r1=findcomponentrevision(TestDummySubComponentRevision, ts.id, v1)
println(v1)

println(r1)

In [ ]:
@testset "reading tests" begin
    v1 = findversion(w1.ref_history, w1.tsdb_validfrom, w1.tsw_validfrom)
    
    @test findcomponentrevision(TestDummyComponentRevision, t.id, v1)[1].description == "blue"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v1)[1].description == "green"
    
    v2 = findversion(w2.ref_history, w2.tsdb_validfrom, w2.tsw_validfrom)
    
    @test findcomponentrevision(TestDummyComponentRevision, t.id, v2)[1].description == "yellow"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v2)[1].description == "green"
    
    v2a = findversion(w2.ref_history, w2.tsdb_validfrom, w2.tsw_validfrom - Dates.Second(1))
    
    @test findcomponentrevision(TestDummyComponentRevision, t.id, v2a)[1].description == "blue"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v2a)[1].description == "green"
    
    
    v3 = findversion(w3.ref_history, w3.tsdb_validfrom, w3.tsw_validfrom)
    
    @test findcomponentrevision(TestDummyComponentRevision, t.id, v3)[1].description == "red"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v3)[1].description == "green"
    
    v3a = findversion(w3.ref_history, w3.tsdb_validfrom, w3.tsw_validfrom - Dates.Second(1))
    
    @test findcomponentrevision(TestDummyComponentRevision, t.id, v3a)[1].description == "blue"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v3a)[1].description == "green"
    
    end

Testing pending transactions and rollback

In [ ]:
w4 = Workflow(
    ref_history = w1.ref_history,
    tsw_validfrom = ZonedDateTime(2017, 11, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
tr4 = copy(tr3)
tr4.description = "green"
t1=TestDummyComponent()
tr5=TestDummyComponentRevision(description="pink")

# @test tr3.ref_invalidfrom==maxVersion

update_entity!(w4)
update_component!(tr3, tr4, w4)
println(tr4)
create_component!(t1, tr5, w4)
println(tr5)

In [ ]:
@testset "pending transaction tests" begin
    v4=find(Version,SQLWhereExpression("id=?", w4.ref_version))[1].id
    @test findcomponentrevision(TestDummyComponentRevision, t.id, v4)[1].description == "green"
    @test findcomponentrevision(TestDummyComponentRevision, t1.id, v4)[1].description == "pink"
    @test w4.ref_version == tr3.ref_invalidfrom
    @test w4.ref_version == tr4.ref_validfrom
    @test maxVersion == tr4.ref_invalidfrom
    @test w4.ref_version == tr5.ref_validfrom
end
println(tr4)
println(tr3)


In [ ]:
b
